### Quick intro to LlamaIndex  
Sources: [1](https://lmy.medium.com/comparing-langchain-and-llamaindex-with-4-tasks-2970140edf33), [2](https://docs.llamaindex.ai/en/stable/), [3](https://github.com/run-llama/llama_index), [4](https://nanonets.com/blog/llamaindex/)  

LlamaIndex is a "data framework" to help you build LLM apps. It provides the following tools:

+ Offers data connectors to ingest your existing data sources and data formats (APIs, PDFs, docs, SQL, etc.).
+ Provides ways to structure your data (indices, graphs) so that this data can be easily used with LLMs.
+ Provides an advanced retrieval/query interface over your data: Feed in any LLM input prompt, get back retrieved context and knowledge-augmented output.
+ Allows easy integrations with your outer application framework (e.g. with LangChain, Flask, Docker, ChatGPT, anything else).
+ LlamaIndex provides tools for both beginner users and advanced users.  

The high-level API allows beginner users to use LlamaIndex to ingest and query their data in 5 lines of code.  
The lower-level APIs allow advanced users to customize and extend any module (data connectors, indices, retrievers, query engines, reranking modules), to fit their needs.  

LlamaIndex provides the following tools:
+ Data connectors ingest your existing data from their native source and format. These could be APIs, PDFs, SQL, and (much) more.
+ Data indexes structure your data in intermediate representations that are easy and performant for LLMs to consume.
+ Engines provide natural language access to your data. For example:
+ Query engines are powerful retrieval interfaces for knowledge-augmented output.
+ Chat engines are conversational interfaces for multi-message, “back and forth” interactions with your data.
+ Data agents are LLM-powered knowledge workers augmented by tools, from simple helper functions to API integrations and more.
+ Application integrations tie LlamaIndex back into the rest of your ecosystem. This could be LangChain, Flask, Docker, ChatGPT, or… anything else!  

In [0]:
!pip install -q openai==0.27.0
#!pip install -qU llama-index            # Just the core components
!pip install llama-index[local_models] # Installs tools useful for private LLMs, local inference, and HuggingFace models
#!pip install llama-index[postgres]     # Is useful if you are working with Postgres, PGVector or Supabase
#!pip install llama-index[query_tools]  # Gives you tools for hybrid search, structured outputs, and node post-processing
# !pip install google-generativeai  #PALM
!pip install -qU pypdf
!pip install -qU docx2txt
!pip install -qU sentence-transformers
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Looking in indexes: https://artifacts.rbi.tech/artifactory/api/pypi/pypi-group/simple/
  Using cached https://artifacts.rbi.tech/artifactory/api/pypi/pypi-group/packages/packages/53/c6/dd12ba91d4a8671d84a40f2469132219cf1dedb7a3da7a73583b0335cc27/llama_index-0.9.46-py3-none-any.whl (15.9 MB)
  Using cached https://artifacts.rbi.tech/artifactory/api/pypi/pypi-group/packages/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached https://artifacts.rbi.tech/artifactory/api/pypi/pypi-group/packages/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.w

In [0]:
import os
import sys
import shutil
import glob
import logging
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
#import tiktoken
#from funcy import lcat, lmap, linvoke
#from IPython.display import display, Markdown
import openai

#OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]  #It has to be defined before importing LlamaIndex modules

## LlamaIndex LLMs
#from openai import OpenAI
#from openai import AzureOpenAI
from llama_index.llms import AzureOpenAI
from llama_index.llms import ChatMessage
#from llama_index.llms import Ollama
#from llama_index.llms import PaLM

## LlamaIndex Embeddings
from llama_index.embeddings import OpenAIEmbedding
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index.embeddings import resolve_embed_model

## Llamaindex readers 
from llama_index import SimpleDirectoryReader
#from llama_index import ResponseSynthesizer

## LlamaIndex Index Types
#from llama_index import GPTListIndex             
from llama_index import VectorStoreIndex
#from llama_index import GPTVectorStoreIndex  
#from llama_index import GPTTreeIndex
#from llama_index import GPTKeywordTableIndex
#from llama_index import GPTSimpleKeywordTableIndex
#from llama_index import GPTDocumentSummaryIndex
#from llama_index import GPTKnowledgeGraphIndex
#from llama_index.indices.struct_store import GPTPandasIndex
#from llama_index.vector_stores import ChromaVectorStore

## LlamaIndex Context Managers
from llama_index import ServiceContext
from llama_index import StorageContext
from llama_index import load_index_from_storage
from llama_index import set_global_service_context
from llama_index.response_synthesizers import get_response_synthesizer
#from llama_index import LLMPredictor

## LlamaIndex Callbacks
#from llama_index.callbacks import CallbackManager
#from llama_index.callbacks import LlamaDebugHandler


## Defining LLM Model
llm_option = "OpenAI"
if llm_option == "OpenAI":
    openai.api_type = "azure"
    azure_endpoint = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
    #azure_endpoint = "https://chatgpt-summarization.openai.azure.com/"
    openai.api_version = "2023-07-01-preview"
    openai.api_key = os.environ["OPENAI_API_KEY"]
    deployment_name = "model-gpt-35-turbo"
    openai_model_name = "gpt-35-turbo"
    llm = AzureOpenAI(api_key=openai.api_key,
                      azure_endpoint=azure_endpoint,
                      model=openai_model_name,
                      engine=deployment_name,
                      api_version=openai.api_version,
                      )
elif llm_option == "Local":  #https://docs.llamaindex.ai/en/stable/module_guides/models/llms.html and https://docs.llamaindex.ai/en/stable/module_guides/models/llms/local.html
    print("Make sure you have installed Local Models - !pip install llama-index[local_models]")
    llm = Ollama(model="mistral", request_timeout=30.0)
else:
    raise ValueError("Invalid LLM Model")

## Defining Embedding Model
emb_option = "Local"
if emb_option == "OpenAI":
    embed_model_name = "text-embedding-ada-002"
    embed_model_deployment_name = "model-text-embedding-ada-002"
    embed_model = AzureOpenAIEmbedding(model=embed_model_name,
                                       deployment_name=embed_model_deployment_name,
                                       api_key=openai.api_key,
                                       azure_endpoint=azure_endpoint)
elif emb_option == "Local":
    embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")   ## bge-m3 embedding model
else:
    raise ValueError("Invalid Embedding Model")

## Logging Optionals
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Text Completion Example

In [0]:
resp = llm.complete("Paul Graham is ")
print(resp)

a computer scientist, entrepreneur, and essayist. He is best known as the co-founder of the startup accelerator Y Combinator, which has helped launch companies such as Dropbox, Airbnb, and Reddit. Graham is also the author of several influential essays on technology, startups, and programming, including "Hackers and Painters" and "The Age of the Essay." He has been a vocal advocate for the importance of creativity, curiosity, and independent thinking in the tech industry.


#### Chat Example Example

In [0]:
messages = [ChatMessage(role="system", content="You are a pirate with a colorful personality"),
            ChatMessage(role="user", content="What is your name"),
            ]
resp = llm.chat(messages)
print(resp)

assistant: My name is Captain Rainbowbeard!


### Quickstart: Implementing a RAG Pipeline:

![](https://docs.llamaindex.ai/en/stable/_images/basic_rag.png)

### Examining Documents Folder

In [0]:
DOCS_DIR = "../../Data/txt/"
docs = os.listdir(DOCS_DIR)
docs = [d for d in docs] # if d.endswith(".txt")]
docs.sort()
for doc in docs:
    print(doc)

Pride_Prejudice.txt
RomeoJuliet.txt


#### Setting the Service Context

In [0]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
set_global_service_context(service_context)

### Creating the Vector Store

In [0]:
PERSIST_DIR = "/Workspace/ds-academy-research/LLamaIndex_quick/"

In [0]:
if os.path.exists(PERSIST_DIR):
    shutil.rmtree(PERSIST_DIR)
print(f"Creating Directory {PERSIST_DIR}")
os.mkdir(PERSIST_DIR)

if os.listdir(PERSIST_DIR) == []:
    print("Loading Documents...")
    documents = SimpleDirectoryReader(DOCS_DIR).load_data()
    print("Creating Vector Store...")
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    print("Persisting Vector Store...")
    index.storage_context.persist(persist_dir=PERSIST_DIR)

Creating Directory /Workspace/ds-academy-research/LLamaIndex_quick/
Loading Documents...
Creating Vector Store...
Persisting Vector Store...


### Reading from existing Vector Store

In [0]:
print("Reading from Vector Store...")
storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
index = load_index_from_storage(storage_context)

Reading from Vector Store...


In [0]:
index.ref_doc_info

{'82aab630-1a18-4b16-835b-ab7cb2e41713': RefDocInfo(node_ids=['7eb9022c-8e3d-4632-9753-54726bb5608d', '519ac749-cffc-4c9b-813d-22911f172b30', '349c5c51-ff3d-406d-9d37-5f70e8f65b17', 'e56ea9d0-485d-42e6-bc80-87338fb562ff', '222ae7a1-f066-46c0-aa7f-9177fab2c63a', 'edabf752-ab30-4e6a-804b-7dff5758a46f', 'c93e0f5b-bc66-4c63-b444-0b5d2264a088', 'd5fb2ef5-c58c-4fb1-95e7-fc0a5c13c818', '29b7acd1-0c9f-4b06-92cf-68ad7fb30de5', '14b000ee-da63-4101-add9-7000d55a0301', '8f900362-9cfa-4476-826a-d060240737d8', 'fae69770-5903-49e4-9667-7622b571d48f', '7311f479-18d8-4c73-a9fc-df010b46d4b6', 'd1b9a07a-3a3c-4093-85d5-a466e0bdd74d', 'fd56df33-f021-4ed7-a659-d185c799c925', '4cc44803-1a45-40bb-93a7-5e72fe5ecb94', '6902d789-5202-426e-92c7-40c81c1af8ec', '0d85bcaa-5905-4480-a4a4-72f3fc0ff278', '3af3ddb2-4772-4114-8ce9-3e3078350ad6', 'a065e396-dde9-454f-bd13-5aa5911dee76', 'cd9ba6fc-0a6b-4870-b2c7-2e3e1b167eb1', '493d06c9-84ae-4c26-9848-8b754c2be9ce', 'b6193665-6d10-44c9-95bb-ed65c52c04d2', 'c652906c-6bc3-44e

### Querying your data  

In [0]:
query_engine = index.as_query_engine(retriever_mode="embedding", response_mode="accumulate", verbose=True)

In [0]:
response = query_engine.query("Who was Romeo?")
print(response)

Response 1: Romeo was a person who had killed Tybalt and was now being advised by Friar Lawrence to go to Mantua until they could find a time to reconcile his friends, beg pardon of the Prince, and call him back with more joy. He was also concerned about Juliet's well-being and wanted to know how she was doing.
---------------------
Response 2: Romeo was a person who came to the same monument where Juliet was buried after hearing the news of her death. He had bought poison from a poor apothecary and came to the vault to die with Juliet.


In [0]:
response = query_engine.query("Who did the proofreading of Pride and Prejudice?")
print(response)

Response 1: There is no information provided in the given context about who did the proofreading of Pride and Prejudice.
---------------------
Response 2: The Online Distributed Proofreading Team at http://www.pgdp.net did the proofreading of Pride and Prejudice.


In [0]:
response = query_engine.query("Who is the publisher of Pride and Prejudice?")
print(response)

Response 1: The publisher of Pride and Prejudice is George Allen.
---------------------
Response 2: The context information does not provide the name of the publisher of Pride and Prejudice.


### Chat with your Data  

[Available Chat Modes](https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/usage_pattern.html)  
+ best - Turn the query engine into a tool, for use with a ReAct data agent or an OpenAI data agent, depending on what your LLM supports. OpenAI data agents require gpt-3.5-turbo or gpt-4 as they use the function calling API from OpenAI.
+ condense_question - Look at the chat history and re-write the user message to be a query for the index. Return the response after reading the response from the query engine.
+ context - Retrieve nodes from the index using every user message. The retrieved text is inserted into the system prompt, so that the chat engine can either respond naturally or use the context from the query engine.
+ condense_plus_context - A combination of condense_question and context. Look at the chat history and re-write the user message to be a retrieval query for the index. The retrieved text is inserted into the system prompt, so that the chat engine can either respond naturally or use the context from the query engine.
+ simple - A simple chat with the LLM directly, no query engine involved.
+ react - Same as best, but forces a ReAct data agent.
+ openai - Same as best, but forces an OpenAI data agent.

In [0]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)
chat_engine.reset()

In [0]:
response = chat_engine.chat("What did happen to Romeo?")
print(response)
response = chat_engine.chat("When was that?")
print(response)
response = chat_engine.chat("What kind of poison?")
print(response)

Querying with: Can you tell me what happened to Romeo?
Romeo bought poison from a poor apothecary and went to the vault to die and lie with Juliet.
Querying with: When did Romeo buy poison from a poor apothecary and go to the vault to die and lie with Juliet?
According to the letter that Romeo's man Balthasar brought to the Prince, Romeo bought poison from a poor 'pothecary and went to the vault to die and lie with Juliet. The exact time is not mentioned in the given context.
Querying with: What kind of poison did Romeo buy from the poor apothecary before going to the vault to die with Juliet?
According to the provided context, Romeo bought a poison from a poor 'pothecary before going to the vault to die with Juliet. The specific kind of poison is not mentioned.


### Using  REPL interface  

In [0]:
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Can you tell me what happened to Romeo?

Querying with: What is the update on Romeo's situation?
Assistant: The given context does not provide information on the current situation of Romeo. The last mention of Romeo is when he buys poison from an apothecary and goes to Juliet's grave to use it. The context then shifts to the aftermath of the tragic events involving Romeo and Juliet.



Human:  What kind of poison?

Querying with: What kind of poison did Romeo buy from the apothecary before going to Juliet's grave?
Assistant: Romeo bought a poison from a poor 'pothecary before going to the vault to die and lie with Juliet. The type of poison is not specified in the given context information.



Human:  Who is the publisher of Pride and Prejudice?

Querying with: What is the name of the publisher of Pride and Prejudice?
Assistant: The name of the publisher of Pride and Prejudice is George Allen.



Human:  exit